In [ ]:
# nbi:hide_in
##########################
# Created on Mar 2020
# @author: juans
##########################

In [3]:
# nbi:hide_in
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline

In [ ]:
# nbi:hide_in
tSlider = widgets.IntSlider(
    value=5,
    min=2,
    max=10,
    step=1,
    description='Sampling Period',
#     continuous_update=False
)

xOffsetSlider = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='xOffset',
#     continuous_update=False,
    readout_format='.1f',
)

xZoomSlider = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='xZoom',
#     continuous_update=False,
    readout_format='d',
)

In [ ]:
# nbi:hide_in
N = 1
frac = 2520 / N
na = np.arange(0, N, 1/frac)
numFreqs = 200
f = np.arange(numFreqs) + 1
g = (1 - f / (2 * numFreqs))
phi = (np.pi * f)**1.08

x = np.imag(np.inner(g, np.exp(1j * 2 * np.pi * np.outer(f, na).T/N + 1j * phi)))
x = x + 0.5
x = x / np.max(np.abs(x))
wa = na - 0.5

In [2]:
# nbi:hide_in
def plot(T, xOffset, xZoom):
    fs = 1/T

    y = np.zeros_like(na)
    y[::T] = 1
    n = na[::T]
    z = x * y

    Z = np.fft.fftshift(np.fft.fft(z)) / len(z)
    S = np.zeros_like(wa)
    S[np.abs(wa) <= fs/2] = 1
    s = np.real(np.fft.fftshift(np.fft.ifft(np.fft.fftshift(S))))
    scaleFact = 1 / np.max(np.abs(s))
    s *= scaleFact
    S *= scaleFact
    R = S * Z
    r = np.real(np.fft.ifft(np.fft.fftshift(R))) * len(z)

    plt.figure(1, figsize=(16, 12))
    plt.subplot(321)
    ml, sl, bl = plt.stem(n, z[::T], use_line_collection=True)
    plt.grid(True)
    plt.ylim([-1.1, 1.1])
    plt.xlim( xOffset + np.array([0, 1]) / (2.0**xZoom))
    plt.title("Waveform")

    plt.subplot(323)
    plt.plot(na, s, lw=1)
    plt.grid(True)
    plt.ylim([-1.1, 1.1])
    plt.xlim( 0.5 + (np.array([0, 1]) - 0.5) / (2.0**xZoom))
    
    plt.subplot(325)
    plt.plot(na, r)
    plt.grid(True)
    plt.ylim([-1.1, 1.1])
    plt.xlim( xOffset + np.array([0, 1]) / (2.0**xZoom))
    plt.xlabel('Time (au)')

    plt.subplot(322)
    plt.plot(wa, np.abs(Z))
    plt.axvspan(-fs/2, fs/2, alpha=0.1, color='k')
    plt.grid(True)
    plt.autoscale(tight=True, axis='x')
    plt.title("Spectrum")
    plt.plot([-fs/2, -fs/2], [0, np.max(np.abs(Z))], 'r', label='NyquistLimit')
    plt.plot([fs/2, fs/2], [0, np.max(np.abs(Z))], 'r')
    plt.legend()

    plt.subplot(324)
    plt.plot(wa, np.abs(S))
    plt.axvspan(-fs/2, fs/2, alpha=0.1, color='k')
    plt.grid(True)
    plt.autoscale(tight=True, axis='x')

    plt.subplot(326)
    plt.plot(wa, np.abs(R))
    plt.axvspan(-fs/2, fs/2, alpha=0.1, color='k')
    plt.grid(True)
    plt.autoscale(tight=True, axis='x')
    plt.xlabel('Frequency (au)')
    
widgets.interactive(plot, T=tSlider, xOffset=xOffsetSlider, xZoom=xZoomSlider)

interactive(children=(IntSlider(value=5, description='Sampling Period', max=10, min=2), FloatSlider(value=0.0,…